# Lillian González Albino   
## Binomios de Involución

A continuación se encuentra el código utilizado para formar conjeturas postuladas en el technical report Agosto 2018. Se documenta todo el procedimiento comenzando con las funciones que se utilizaron para probar que $P(x)=x^m(x^{\frac{p-1}{2}}+a)$ es permutación en $\mathbb{F}_p$, seguido por las conjeturas formadas para buscar condiciones suficientes y necesarias para que $P(x)$ sea involución en $\mathbb{F}_p$. Se documentan los resultados de las conjeturas, pero no se despliegan ya que los resultados contradicen las conjeturas, también se documenta los próximos pasos a seguir. Al final se despliegan los resultados de la última conjetura hecha ya que los resultados van acorde con la conjetura. Esta última conjetura se probará luego en el technical report Agosto 2018.

Verificar conjetura: Sea $(m,p-1)=1$. Si $\eta(a^2-1)=1$, entonces $P(x)=x^m(x^{\frac{p-1}{2}}+a)$ es permutación en $\mathbb{F}_p$

In [2]:
#input: p primo
#output: lista de enteros
#recibe un primo p y devuelve una lista de todas las a que 
#satisfacen que a^2-1 se un residuo cuadratico mod p

def findAs(p):
    listOfAs =[]
    b = quadratic_residues(p)
    for a in range(p):
        if((a^2-1)%p in b):
            if(a != 0 and (a^2-1)%p != 0):
                listOfAs.append(a)
    return listOfAs
    
#input: p primo
#output: lista de enteros
#recibe un primo p y devuelve una lista de todas las m 
#que sean relativamente primas a p-1

def findMs(p):
    listOfMs = []
    for m in range(p):
        if(gcd(m,p-1)==1):
            listOfMs.append(m)
    return listOfMs

#input: primo p, int a, int m
#output: bool
#recibe un primo p, una a y una m y verifica si el polinomio 
#con las dadas variables es de permutacion

def checkifPP(p,a,m):
    A = range(p)
    B = []
    for x in A:
        P = x^m * (x^((p-1)/2) + a)
        B.append(P%p)
    if(set(A) == set(B)):
        return True
    else:
        return False

#input: primo p, lista de enteros lA, lista de enteros lM
#output: void
#recibe un primo p y una lista de as y ms y verifica si todas 
#las combinaciones de as y ms (bajo mod p) son polinomios de 
#permutacion. Despliega las combinaciones (p,m,a) que no sean 
#de permutacion

def checkAllPP(p, lA, lM):
    for a in lA:
        for m in lM:
            if(checkifPP(p,a,m)==False):
                print(p,a,m)

Funciones que devuelven todas las involuciones en $\mathbb{Z}_p$ dado un primo $p$. 

In [3]:
#input: p pirmo, int c
#output: i int
#recibe un p primo y un int c, calcula alpha^i 
#(alpha raiz primitiva de Z_p) y devuelve i 

def findi(p, c):
    alpha = primitive_root(p)
    for i in range(p):
        if((alpha^i)%p == c):
            return i

#input: p primo, int m, int a
#output: bool
#recibe un p primo, int m, int a, y verifica si el binomio 
#con las dadas variables es de involucion
#add verificar puntos fijos > 0

def checkifInv(p,m,a):
    h = (p-1)/2
    d = 0
    for x in range(1,p+1):
        Px = ( x^m * (x^h + a) )%p
        PPx = ( Px^m * (Px^h + a) )%p
        if(Px == x):
            d += 1
        if(PPx == 0):
            PPx = p
        if(PPx != x):
            return False
    if(d > 0):
        return True
    else:
        return False

#input: p primo
#output: lista (int, int, int)
#recibe un primo p y calcula todas las involucione de esa p 
#y devuelve una lista con elementos tuplas (p,m,a) que hacen 
#el binomio una involucion

def findAllInv(p):
    lM = findMs(p)
    lA = findAs(p)
    lInv = []
    for m in lM:
        for a in lA:
            if(checkifInv(p,m,a)):
                lInv.append((m,a))
    return lInv

#input: primo p, int m, int a
#output: int d
#recive un primo p, int m, int a, verifica si es un 
#polinomio de involucion y si lo es devuelve la cantidad 
#de puntos fijos del mismo

def countFixedPoints(p,m,a):
    if(checkifInv(p,m,a)==False):
        return -1
    else:
        d = 0
        for x in range(1,p):
            Px = ( x^m * (x^((p-1)/2) + a) )%p
            if( x == Px ):
                d += 1
        return d

#input: p primo, int m,a,d
#output: int k
#recibe p primo, int m,a,d y devuelve k = ((m+1)*d)/(p-1)
#note: solo funciona cuando las variables dadas son 
#involucion del polinomio

def findK(p,m,a,d):
    k = ((m+1)*d)/(p-1)
    return k

#input: p primo
#output: lista (int,int,int,int)
#recibe un primo p, calcula todas las involuciones, 
#sus puntos fijos, y su k = ((m+1)*d)/(p-1) y los 
#devuelve en una lista de tuplas conteniendo las invulociones

def findAllKandD(p):
    lI = findAllInv(p)
    lInvDK = []
    for elem in lI:
        m = elem[0]
        a = elem[1]
        d = countFixedPoints(p,m,a)
        if( d != 0 ):
            k = findK(p,m,a,d)
            lInvDK.append((p,m,a,d,k))
    return lInvDK

Funcion que devuelve todas las involuciones de $\mathbb{Z}_p$ para distintos primos $p$

In [4]:
#input: int frm, int n, int fixpoint (opcional)
#output: void
#verifica todas las involuciones desde el proximo primo despues de 
#frm y los proximos n primos despues de from y despliega las 
#combiaciones (p,m,a,d,k) de involuciones. Si el argumento fixpoint 
#esta dado, solo despliega las involvuciones con fixpoint cantidad 
#de puntos fijos (sin incluir el 0)

def findInvolution(frm, n, fixpoint = None):
    P = Primes()
    p = frm -1
    for i in range(n):
        p = P.next(p)
        for elem in findAllKandD(p):
            d = elem[3]
            if(fixpoint is None):
                print(elem)
            else:
                if(d == fixpoint):
                    print(elem)

Función complementaria que devuelve todas las $i$ tal que $\alpha^i=c$ para una lista dada de c's  
($\alpha$ raiz primitida de $\mathbb{Z}_p$)

In [5]:
#input: p primo, lista int lC
#output: void
#recibe un primo p y una lista de cs, despliega todas las i tal 
#que alpha^i==c (alpha raiz primitida de Z_p)

def findAlli(p, lC):        
    for x in lC:            
        print(x,findi(p, x))

Funciones complementaria que clasifica a $p$ de la forma $4k+1$ o $4k+3$ ($k \in \mathbb{Z}$)

In [6]:
#input: p primo
#output: 1,3 
#coge un primo p y devuelve 1 si p=4k+1 y, 3 si p=4k+3
def findPform(p):
    pForm = 0
    if(((p-1) % 4) == 0):
        pForm = 1
    if(((p-3) % 4) == 0):
        pForm = 3
    return pForm

#input: p primo
#output: coge los primeros 10 primos desde p y 
#devuelve 1 si p=4k+1 y, 3 si p=4k+3 para cada p
def pForms(startP):
    P = Primes()
    p = startP - 1
    for i in range(10):
        p = P.next(p)
        print(p, findPform(p))

Tratanto de probar conjeturas pasadas sobre $P(x)$ se encontró que para $m^2 = \frac{p-1}{2} (impar) + 1$ no se cumplen. Esta función es para encontrar $m^2$ que satisfacen estas condiciones para poder estudiarlas.

In [7]:
#buscar involuciones 
#buscar m^2 que cumplan con condiciones dadas 
def m_squared(p):
    invs = findAllInv(p)
    for inv in invs:
        #print(inv)
        m = inv[0]
        a = inv[1]
        var = (2*(m^2-1))/(p-1)
        print(m,a,(m^2)%p,var)

Fijando diferentes $p$ se encontró con la función anterior que el único primo $p<90$ que satisface las condiciones dadas es $p=17$. Estudiaremos más a fondo este primo.

Función complementaria que verifica proposiciones (pasadas como parámetros) para diferentes primos $p$

In [8]:
#input: p primo, int amount, funcion func (que su output sea void)
#output: void
#Despliega el output de la funcion (func) para una 
#cantidad (amount) de primos
def checkpropDifferentP(p, amount, func):
    P = Primes()
    for i in range(amount):
        func(p)
        p = P.next(p)

Verificar conjetura 1: Sea $(m,q-1)=1$. Si $\eta(a^2-1)=1$ y $m^2 \equiv 1 \ (mod \ \frac{q-1}{2})$ y $(a+1)^{m+1} = 1$, entonces P(x) es involución en $\mathbb{F}_q$

Todo reducido mod $p$

In [9]:
#input: p primo
#output: void
# verifica la conjetura: gcd(m,p-1)==1 y a^2-1 r.c. y 
#m^2 congr 1 (mod) (p-1)/2 y (a+1)^(m+1) % p == 1  => involucion
def checkprop1(p):
    lA = findAs(p)
    for m in range(p):
        if(gcd(m, p-1)==1):
            halfp = (p-1)/2
            if(m^2 % halfp == 1):
                for a in lA:
                    plusa = a+1
                    minusa = a-1
                    expo = m+1
                    if(plusa^expo % p == 1):
                        print(p,m,a,checkifInv(p,m,a))

Se encontró en el output anterior, salieron todas las involuciones, sin embargo, también salieron otras combinaciones de (p,m,a) que no hacen $P(x)$ una involución.

Verificar conjetura 2: Sea $(m,q-1)=1$. Si $\eta(a^2-1)=1$ y $m^2 \equiv 1 \ (mod \ \frac{q-1}{2})$ y $(a+1)^{m+1} = 1$ y $(a-1)^{m+1} = 1$, entonces P(x) es involución en $\mathbb{F}_q$

Todo reducido mod $p$

In [10]:
#input: p primo
#output: void
# verifica la conjetura: gcd(m,p-1)==1 y a^2-1 r.c. y 
#m^2 congr 1 (mod) (p-1)/2 y (a+1)^(m+1) == 1  y 
#(a-1)^(m+1) == 1 => involucion
def checkprop2(p):
    lA = findAs(p)
    for m in range(p):
        if(gcd(m, p-1)==1):
            halfp = (p-1)/2
            if(m^2 % halfp == 1):
                for a in lA:
                    plusa = a+1
                    minusa = a-1
                    expo = m+1
                    if(plusa^expo % p == 1):
                        if(minusa^expo % p == 1):
                            print(p,m,a,checkifInv(p,m,a))

Se encontró en el output anterior que, al igual que en la conjetura 1, salen "todas" las involuciones pero también salen otras combinaciones de (p,m,a) que no hacen a $P(x)$ una involución. Con la excepción del p=17 en el cual solo salen mitad de las permutaciones.

Observacion: Para todas las involuciones $(a+1)^{m+1}=1$ pero para solo algunas $(a-1)^{m+1}=1$.

Modificación conjetura 2: Sea $(m,q-1)=1$. Si $\eta(a^2-1)=1$ y $m^2 \equiv 1 \ (mod \ \frac{q-1}{2})$ y $(a+1)^{m+1} = 1$ y [$(a-1)^{m+1} = 1$ o $(a-1)^{m+1} = -1$], entonces P(x) es involución en $\mathbb{F}_q$

Todo reducido mod $p$

In [11]:
#input: p primo
#output: void
# verifica la conjetura: gcd(m,p-1)==1 y a^2-1 r.c. y 
#m^2 congr 1 (mod) (p-1)/2 y (a+1)^(m+1) == 1  y (a-1)^(m+1) == 1 
#o -1 => involucion
def checkprop25(p):
    lA = findAs(p)
    for m in range(p):
        if(gcd(m, p-1)==1):
            halfp = (p-1)/2
            if(m^2 % halfp == 1):
                for a in lA:
                    plusa = a+1
                    minusa = a-1
                    expo = m+1
                    if(plusa^expo % p == 1):
                        v = -1 % p
                        if(minusa^expo % p == 1):
                            print(p,m,a,checkifInv(p,m,a))
                        if(minusa^expo % p == v):
                            print(p,m,a,checkifInv(p,m,a))

Se encontró en el output anterior que con las condiciones dadas salen todas las involuciones (incluyendo p=17) pero también salen combinaciones (p,m,a) que no hacen a P(x) una involución

Próximo paso: verificar otras condiciones que pueden reducir el output para que solamente salgan las involuciones. Aparte, averiguar cuándo $(a-1)^{m+1}$ es igual a 1 y cuándo es igual a -1.

Verificar conjetura 3: Sea $(m,q-1)=1$. Si $\eta(a^2-1)=1$ y $m^2 \equiv 1 \ (mod \ \frac{q-1}{2})$ y $(a+1)^{m+1} = 1$ y $(a+1)^{\frac{p-1}{2}} = 1$ y [$(a-1)^{m+1} = 1$ o $(a-1)^{m+1} = -1$], entonces P(x) es involución en $\mathbb{F}_q$ 

Todo reducido mod $p$

In [12]:
#input: p primo
#output: void
# verifica la conjetura: gcd(m,p-1)==1 y a^2-1 r.c. 
#y m^2 congr 1 (mod) (p-1)/2 y (a+1)^(m+1) == 1  y (a-1)^(m+1) == 1 
#o -1 y (a+1)^((p-1)/2) => involucion
def checkprop3(p):
    lA = findAs(p)
    for m in range(p):
        if(gcd(m, p-1)==1):
            halfp = (p-1)/2
            if(m^2 % halfp == 1):
                for a in lA:
                    plusa = a+1
                    minusa = a-1
                    expo = m+1
                    halfp = (p-1)/2
                    if(plusa^expo % p == 1):
                        if(plusa^halfp % p == 1):
                            v = -1 % p
                            if(minusa^expo % p == 1):
                                print(p,m,a,checkifInv(p,m,a))
                            if(minusa^expo % p == v):
                                print(p,m,a,checkifInv(p,m,a))

Se encontró en el output anterior que para $p=4k+3$ salen todas las involuciones y no sobran ningunos pares ordenados (p,m,a) que no hagan a $P(x)$ involución. Es decir, tenemos condiciones suficientes y necesarias para que $P(x)$ sea involución. Sin embargo, para $p=4k+1$, aunque salen todas las involcuiones todavía salen otros pares ordenados (p,m,a) que no hacen a $P(x)$ una involución.

Próximos pasos:   
1) Ver si para $p=4k+3$ se puede eliminar el caso de $(a-1)^{m+1}=-1$ y si con $(a-1)^{m+1}=1$ junto con las otras dos condiciones es suficiente y necesario para generar las involuciones.   
2) Buscar otra condición adicional para $p=4k+1$ que haga que sean suficientes y necesarias para que sea involución.  
3) (De las observaciones de la conjetura 2 modificada) Ver cuándo $(a-1)^{m+1}=-1$ y cuándo $(a-1)^{m+1}=1$

Verificar conjetura 4: Sea $(m,q-1)=1$ y $p=4k+3$. Si $\eta(a^2-1)=1$ y $m^2 \equiv 1 \ (mod \ \frac{q-1}{2})$ y $(a+1)^{m+1} = 1$ y $(a+1)^{\frac{p-1}{2}} = 1$ y $(a-1)^{m+1} = 1$, entonces P(x) es involución en $\mathbb{F}_q$ 

Todo reducido mod $p$

In [13]:
#input: p primo
#output: void
# verifica la conjetura 2: p=4k+3 y gcd(m,p-1)==1 y a^2-1 r.c. 
#y m^2 congr 1 (mod) (p-1)/2 y (a+1)^(m+1) == 1  y (a-1)^(m+1) == 1 
#y (a+1)^((p-1)/2)=1 => involucion
def checkprop4(p):
    if(findPform(p)==3):
        lA = findAs(p)
        for m in range(p):
            if(gcd(m, p-1)==1):
                halfp = (p-1)/2
                if(m^2 % halfp == 1):
                    for a in lA:
                        plusa = a+1
                        minusa = a-1
                        expo = m+1
                        halfp = (p-1)/2
                        if(plusa^expo % p == 1):
                            if(plusa^halfp % p == 1):
                                if(minusa^expo % p == 1):
                                    print(p,m,a,checkifInv(p,m,a))
    else:
        print("p=4k+1")

Se encontró con la función anterior que las condiciones dadas son suficiente y necesarias para que $P(x)$ esa involución con $p=4k+3$  

Próximos pasos:  
1) Buscar otra condición adicional para $p=4k+1$ que haga que sean suficientes y necesarias para que sea involución.  
2) (de las observaciones de la conjetura 2 modificada) Ver cuándo $(a-1)^{m+1}=-1$ y cuándo $(a-1)^{m+1}=1$

Resultados de la conjetura 4

In [15]:
#lista de primos de la forma p=4k+3
p3 = [7,11,19,23,31,43,47,59,67,71,79,83]
for p in p3:
    checkprop4(p)

(7, 5, 3, True)
(11, 9, 2, True)
(11, 9, 4, True)
(19, 17, 5, True)
(19, 17, 6, True)
(19, 17, 8, True)
(19, 17, 10, True)
(23, 21, 2, True)
(23, 21, 3, True)
(23, 21, 5, True)
(23, 21, 7, True)
(23, 21, 17, True)
(31, 19, 3, True)
(31, 29, 3, True)
(31, 29, 6, True)
(31, 29, 8, True)
(31, 29, 9, True)
(31, 29, 15, True)
(31, 29, 17, True)
(31, 29, 19, True)
(43, 41, 5, True)
(43, 41, 10, True)
(43, 41, 12, True)
(43, 41, 14, True)
(43, 41, 15, True)
(43, 41, 16, True)
(43, 41, 22, True)
(43, 41, 24, True)
(43, 41, 37, True)
(43, 41, 39, True)
(47, 45, 2, True)
(47, 45, 3, True)
(47, 45, 5, True)
(47, 45, 7, True)
(47, 45, 8, True)
(47, 45, 13, True)
(47, 45, 15, True)
(47, 45, 17, True)
(47, 45, 26, True)
(47, 45, 33, True)
(47, 45, 35, True)
(59, 57, 2, True)
(59, 57, 4, True)
(59, 57, 6, True)
(59, 57, 8, True)
(59, 57, 16, True)
(59, 57, 18, True)
(59, 57, 20, True)
(59, 57, 21, True)
(59, 57, 26, True)
(59, 57, 27, True)
(59, 57, 28, True)
(59, 57, 47, True)
(59, 57, 50, True)
(59

Se puede verificar que en el output anterior salen todas las involuciones de $P(x)$ para $p=4k+3$